In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['Fantrax-players.csv']


In [35]:
players = pd.read_csv('../input/Fantrax-players.csv')

In [36]:
playersGrouped = players[players['Status']!='FA'].groupby(by='Status',as_index=False).mean()
playersGrouped.drop(['Rk','GP'], axis=1,inplace=True)
labels = playersGrouped.columns[1:]

In [37]:
playersGroupedNorm = playersGrouped.copy()
for label in labels:
    if label.endswith('%'):
        continue
    playersGroupedNorm[label]-=playersGroupedNorm[label].min()
    playersGroupedNorm[label]/=playersGroupedNorm[label].max()
    if label == 'TO':
        playersGroupedNorm[label]=1-playersGroupedNorm[label]
playersGroupedNorm.describe()

,Score,FG%,3PTM,FT%,PTS,REB,AST,ST,BLK,TO
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,0.580756,0.466679,0.414997,0.775666,0.679267,0.446902,0.350684,0.525112,0.629564,0.577887
std,0.252566,0.013600,0.291510,0.020145,0.288768,0.289050,0.299243,0.306043,0.267500,0.286106
min,0.000000,0.444692,0.000000,0.749077,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.457888,0.457904,0.201220,0.758212,0.610432,0.252488,0.153557,0.302455,0.517781,0.460236
50%,0.627537,0.466868,0.402003,0.776654,0.713272,0.405295,0.299595,0.546875,0.658606,0.595276
75%,0.717115,0.472526,0.591463,0.788547,0.892374,0.656716,0.523279,0.703125,0.806543,0.722047
max,1.000000,0.498538,1.000000,0.811286,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def f(x):
    return x

import seaborn as sns
import matplotlib.pyplot as plt

from ipywidgets import Checkbox, interactive
from IPython.display import display

l = playersGrouped['Status']
chk = [Checkbox(description=a) for a in l]

def updatePlot(**k):
    #print(k)
    teamsToShow = []
    for key in k.keys():
        if k[key] is True:
            teamsToShow.append(key)
    if len(teamsToShow) == 0:
        return
    print(teamsToShow)
    fig=plt.figure(figsize=(12, 10))
    labels=np.array(playersGrouped.columns[1:])
    ax = fig.add_subplot(111, polar=True)
    ax.set_title("Compare")
    i=0
    for team in teamsToShow:
        color='C'+str(i)
        stats=playersGroupedNorm[playersGroupedNorm['Status']==team][labels].values[0]
        #print(stats)
        angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
        # close the plot
        stats=np.concatenate((stats,[stats[0]]))
        angles=np.concatenate((angles,[angles[0]]))

        ax = fig.add_subplot(111, polar=True)
        ax.plot(angles, stats, 'o-', linewidth=2, label=team)
        ax.fill(angles, stats, alpha=0.25, color=color)
        ax.set_thetagrids(angles * 180/np.pi, labels)
        #ax.set_title([name], color=color)
        ax.grid(True)
        i+=1
    legend = ax.legend(loc=1)

interact(updatePlot, **{c.description: c.value for c in chk})


interactive(children=(Checkbox(value=False, description='GFv'), Checkbox(value=False, description='GREEN'), Ch…

<function __main__.updatePlot(**k)>